In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

PARALLEL = 8 # assuming a eight-core machine
ATTRIBUTE = "zeugma"

os.environ['FONDUERDBNAME'] = ATTRIBUTE
os.environ['SNORKELDB'] = 'postgres://weijiechen1994@localhost:5432/' + os.environ['FONDUERDBNAME']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from fonduer import SnorkelSession

session = SnorkelSession()

In [5]:
from fonduer import HTMLPreprocessor, OmniParser

docs_path = 'data/html/'
pdf_path = 'data/pdf/'

max_docs = float('inf')
doc_preprocessor = HTMLPreprocessor(docs_path, max_docs=max_docs)

In [ ]:
# corpus_parser = OmniParser(structural=True, lingual=True, visual=True, pdf_path=pdf_path)
# %time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

In [6]:
from fonduer import Document, Phrase

print("Documents:", session.query(Document).count())
print("Phrases:", session.query(Phrase).count())

Documents: 42
Phrases: 96736


In [45]:
docs = session.query(Document).order_by(Document.name).all()
ld   = len(docs)

train_docs = set()
dev_docs   = set()
test_docs  = set()
splits = (0.8, 0.9)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    print(doc_name)
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint
pprint([x.name for x in train_docs])
pprint([x.name for x in dev_docs])
pprint([x.name for x in test_docs])

v1ch01
v1ch02
v1ch03
v1ch04
v1ch05
v1ch06
v1ch07
v1ch08
v1ch09
v1ch10
v1ch11
v1ch12
v1ch13
v1fold-out-plates1
v1fwd
v1pref
v1shoreline
v2ch01
v2ch02
v2ch03
v2ch04
v2ch05
v2ch06
v2ch07
v2ch08
v2hawari-plates
v2kenrick-plates
v2reynolds-plates
v3ch01
v3ch02
v3ch03
v3ch04
v3ch05
v3ch06
v3ch07
v3ch08
v3ch09
v3ch10
v3ch11
v3ch12
v3ch13
v3ch14
['v1ch02',
 'v2kenrick-plates',
 'v1fwd',
 'v1ch03',
 'v2reynolds-plates',
 'v1pref',
 'v1ch04',
 'v1ch05',
 'v3ch01',
 'v1shoreline',
 'v1ch06',
 'v3ch02',
 'v2ch01',
 'v1ch07',
 'v3ch03',
 'v2ch02',
 'v1ch08',
 'v3ch04',
 'v2ch03',
 'v1ch09',
 'v3ch05',
 'v2ch04',
 'v1ch10',
 'v3ch06',
 'v2ch05',
 'v1ch11',
 'v2ch06',
 'v1ch12',
 'v2ch07',
 'v1ch13',
 'v2ch08',
 'v1fold-out-plates1',
 'v2hawari-plates',
 'v1ch01']
['v3ch08', 'v3ch09', 'v3ch07', 'v3ch10']
['v3ch13', 'v3ch11', 'v3ch14', 'v3ch12']


In [8]:
from fonduer import candidate_subclass

Cata_Matl = candidate_subclass('Cata_Matl', ['cata','matl'])

In [27]:
from fonduer import RegexMatchSpan, DictionaryMatch, LambdaFunctionMatcher, Intersect, Union

# v1ch13
# matl1_rgx = r'terracotta'
# matl2_rgx = r'ceramic'
# matl3_rgx = r'limestone'
# matl4_rgx = r'stone'
# matl5_rgx = r'basalt'
# matl6_rgx = r'clay'
# matl7_rgx = r'ceramic'
# matl8_rgx = r'terracotta'

mat_v1c01_list = ['gold objects' ,'architecture' ,'bullae' ,'ceramic lamps',
             'coins' ,'copper alloy' ,'crowther' ,'glass' ,'inscriptions',
             'ironwork' ,'loom weights' ,'military equipment' ,'mosaics',
             'pottery' ,'quern' ,'spindle whorls' ,'stone mortar' ,'stone sculpture',
             'stone tools' ,'stone vessels' ,'terracotta figurines', 'textiles',
             'transport amphorae' ,'bone' ,'ivory']

mat_v1c13_list = [r'basalt',r'carnelianbead',r'ceramic', r'clay',
                  r'composite',r'copper alloy',r'faience',r'firedclay',
                  r'flint',r'glass',r'iron',r'lead',r'limestone',
                  r'marble',r'metal',r'plaster',r'quartz',r'shell',
                  r'stone',r'terracotta',r'wood']

for rgx in mat_v1c01_list:
    matl_rgx = '|'.join([matl_rgx, rgx])
for rgx in mat_v1c13_list:
    matl_rgx = '|'.join([matl_rgx, rgx]) 
matl_matcher = RegexMatchSpan(rgx=matl_rgx, longest_match_only=False)

In [28]:
from fonduer import RegexMatchSpan, DictionaryMatch, LambdaFunctionMatcher, Intersect, Union

cata_rgx = r'(A|AM|B|BR|C|G|GD|IN|IR|L|LW|M|ML|PT|Q|SM|SS|ST|SV|SW|TC|TX|ZB)[0-9]{1,3}'
cata_matcher = RegexMatchSpan(rgx=cata_rgx, longest_match_only=True)

In [47]:
# from zeugma_space import OmniNgramsCata, OmniNgramsLabref
from fonduer import OmniNgrams

cata_ngrams = OmniNgrams(n_max=1)
matl_ngrams = OmniNgrams(n_max=2)

In [48]:
from fonduer.lf_helpers import *
import re

def cata_matl_filter(c):
    (cata, matl) = c
    flag_same_page = same_page(c)
    flag_same_sentence = same_sentence(c)
    return (flag_same_sentence and flag_same_page)
    return True

candidate_filter = cata_matl_filter

In [49]:
from fonduer import CandidateExtractor

candidate_extractor = CandidateExtractor(Cata_Matl, 
                        [cata_ngrams, matl_ngrams], 
                        [cata_matcher, matl_matcher], 
                        candidate_filter=candidate_filter)

%time candidate_extractor.apply(train_docs, split=0, parallelism=PARALLEL)

CPU times: user 17.1 ms, sys: 103 ms, total: 120 ms
Wall time: 13min 25s


In [50]:
train_cands = session.query(Cata_Matl).filter(Cata_Matl.split == 0).all()
print("Number of candidates:", len(train_cands))

Number of candidates: 3513


In [37]:
for i in train_cands:
#     print(i)
    char_start0 = i.get_contexts()[0].char_start
    char_end0 = i.get_contexts()[0].char_end
    char_start1 = i.get_contexts()[1].char_start
    char_end1 = i.get_contexts()[1].char_end
    print("Objective 0: ", i.get_contexts()[0].sentence.text[char_start0:char_end0+1])
    print("Objective 1: ", i.get_contexts()[0].sentence.text[char_start1:char_end1+1])
    print(i.get_contexts()[0].sentence.document)
    print("Text: ", i.get_contexts()[0].sentence.text)
#     print(i.values)
    print('----------------------')

Objective 0:  AM330
Objective 1:  Quartz
Document b'v2ch03'
Text:  Quartz range is uneven, not even as AM330.
----------------------
Objective 0:  G118
Objective 1:  faience
Document b'v2ch08'
Text:  Even more modest was a finger ring made entirely of faience, with a conical bezel (G118).
----------------------
Objective 0:  G17
Objective 1:  glass
Document b'v2ch08'
Text:  A fragment of a conical beaker (G17) recovered at Zeugma illustrates a decorative technique known as relief-cutting, in which the cutting wheel was used to grind down parts of the exterior surface of a blown-glass blank, leaving other areas raised in relief.
----------------------
Objective 0:  G91
Objective 1:  glass
Document b'v2ch08'
Text:  From the fourth century onwards, lamps became a major compo- nent of Byzantine glass production throughout the east- ern Mediterranean.47A relatively small number of ceramic lamps datable to the fifth and sixth centuries was recovered at Zeugma, suggesting a decline in their u

Objective 0:  AM113
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, copper alloy, glass Catalogued finds: AM90, AM91, AM92, AM93, AM94, AM95, AM96, AM97, AM98, AM99, AM100, AM101, AM102, AM103, AM104, AM105, AM106, AM107, AM108, AM109, AM110, AM111, AM112, AM113, AM114, C90, G48, IR371, IR415, L10, L120, PT341, PT343, PT344, PT356, PT357, PT360, PT362, PT368, PT375, PT378, PT382, PT385, PT430, PT434, PT438, PT456, Q3
----------------------
Objective 0:  AM113
Objective 1:  bone
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, copper alloy, glass Catalogued finds: AM90, AM91, AM92, AM93, AM94, AM95, AM96, AM97, AM98, AM99, AM100, AM101, AM102, AM103, AM104, AM105, AM106, AM107, AM108, AM109, AM110, AM111, AM112, AM113, AM114, C90, G48, IR371, IR415, L10, L120, PT341, PT343, PT344, PT356, PT357, PT360, PT362, PT368, PT375, PT378, PT382, PT385, PT430, PT434, PT438, PT456, Q3
----------------------
Objective 0:  AM113
Objective 1:  i

Objective 0:  AM225
Objective 1:  ceramic
Document b'v1ch13'
Text:  Bulk finds: Mosaic tesserae, ceramic, animal bone Catalogued   finds:   AM224,   AM225,   AM226,   AM227, AM228, AM229, AM230, AM231, AM232, B19, BR60, BR62, BR85 (ML42), BR86 (ML41), BR89
----------------------
Objective 0:  AM225
Objective 1:  bone
Document b'v1ch13'
Text:  Bulk finds: Mosaic tesserae, ceramic, animal bone Catalogued   finds:   AM224,   AM225,   AM226,   AM227, AM228, AM229, AM230, AM231, AM232, B19, BR60, BR62, BR85 (ML42), BR86 (ML41), BR89
----------------------
Objective 0:  AM226
Objective 1:  ceramic
Document b'v1ch13'
Text:  Bulk finds: Mosaic tesserae, ceramic, animal bone Catalogued   finds:   AM224,   AM225,   AM226,   AM227, AM228, AM229, AM230, AM231, AM232, B19, BR60, BR62, BR85 (ML42), BR86 (ML41), BR89
----------------------
Objective 0:  AM226
Objective 1:  bone
Document b'v1ch13'
Text:  Bulk finds: Mosaic tesserae, ceramic, animal bone Catalogued   finds:   AM224,   AM225,   AM226,  

Objective 0:  Q17
Objective 1:  stone
Document b'v3ch06'
Text:  Almost completely preserved in 11 joining and several non- joining fragments (some may be from Q17, perhaps the matching upper grinding stone).
----------------------
Objective 0:  Q23
Objective 1:  stone
Document b'v3ch06'
Text:  The slope of the grinding face on Q23 and Q25 is the wrong angle for an upper stone, since it slopes up to the perforation.
----------------------
Objective 0:  Q10
Objective 1:  stone
Document b'v3ch06'
Text:  The other is Q10, an upper grinding stone.
----------------------
Objective 0:  Q25
Objective 1:  stone
Document b'v3ch06'
Text:  The slope of the grinding face on Q23 and Q25 is the wrong angle for an upper stone, since it slopes up to the perforation.
----------------------
Objective 0:  Q24
Objective 1:  stone
Document b'v3ch06'
Text:  The nearly level slope of the grinding face on Q24 could be suitable for an upper stone, but the level face and the roughness of the perforation sup- por

Objective 0:  IR215
Objective 1:  stone
Document b'v3ch03'
Text:  Looped spikes were either driven into wood or set in stone, and secured by melted lead (see drop hinge ride/looped spike IR215, above).
----------------------
Objective 0:  IR189
Objective 1:  iron
Document b'v3ch03'
Text:  In addition to the large complete items of iron furniture, the finds also include numerous leaf hinges, many of which may have been used for chests or cupboards (IR189-204).
----------------------
Objective 0:  IR28
Objective 1:  ironwork
Document b'v3ch03'
Text:  The ironwork from the courtyard includes a possible ar- rowhead (IR28) and three tools: a smith’s punch (IR37), an axe (IR40), and a socket, possibly for a tool (IR78).
----------------------
Objective 0:  BR20
Objective 1:  iron
Document b'v3ch03'
Text:  These measure 1,620 mm and 153 mm tall, respec- tively, and are made of copper alloy or bronze and are much more highly decorated than this iron example and comparable to the copper alloy c

...

Objective 0:  M13
Objective 1:  limestone
Document b'v1ch13'
Text:  Description: Burnt destruction debris across entire room, on top of mosaic M13, with frequent inclusions of burnt mud-brick, broken limestone, and roof tiles, as well as lens- es of charcoal.
----------------------
Objective 0:  PT489
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Ceramic, animal bone, slag, glass Catalogued finds: PT489, PT500, PT502, PT504, PT505
----------------------
Objective 0:  PT489
Objective 1:  bone
Document b'v1ch13'
Text:  Ceramic, animal bone, slag, glass Catalogued finds: PT489, PT500, PT502, PT504, PT505
----------------------
Objective 0:  PT489
Objective 1:  glass
Document b'v1ch13'
Text:  Ceramic, animal bone, slag, glass Catalogued finds: PT489, PT500, PT502, PT504, PT505
----------------------
Objective 0:  PT500
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Ceramic, animal bone, slag, glass Catalogued finds: PT489, PT500, PT502, PT504, PT505
----------------------
Objective 0:  

Objective 0:  BR5
Objective 1:  iron
Document b'v1ch13'
Text:  Bulk finds: Glass, ceramic, animal bone, iron, copper alloy Catalogued finds: BR5, BR6, IR61, IR443, TX21, TX22, TX23, TX26
----------------------
Objective 0:  SW27
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, glass Catalogued finds: B15, G112, IR152, L203, SW26, SW27
----------------------
Objective 0:  SW27
Objective 1:  bone
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, glass Catalogued finds: B15, G112, IR152, L203, SW26, SW27
----------------------
Objective 0:  SW27
Objective 1:  iron
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, glass Catalogued finds: B15, G112, IR152, L203, SW26, SW27
----------------------
Objective 0:  SW27
Objective 1:  glass
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, glass Catalogued finds: B15, G112, IR152, L203, SW26, SW27
----------------------
Objective 0:  BR6
Objective 1:  Glass
Docu

Objective 0:  IR37
Objective 1:  glass
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron, glass Catalogued finds: IR37, IR100, IR507, Q23, SW18
----------------------
Objective 0:  IR100
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron, glass Catalogued finds: IR37, IR100, IR507, Q23, SW18
----------------------
Objective 0:  IR100
Objective 1:  plaster
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron, glass Catalogued finds: IR37, IR100, IR507, Q23, SW18
----------------------
Objective 0:  IR100
Objective 1:  iron
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron, glass Catalogued finds: IR37, IR100, IR507, Q23, SW18
----------------------
Objective 0:  IR100
Objective 1:  glass
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron, glass Catalogued finds: IR37, IR100, IR507, Q23, SW18
----------------------
Objective 0:  IR507
Objective 1:  Ceramic
Document b'v1ch13'
Text:  Bulk finds: Ceramic, plaster, iron,

Objective 1:  Ceramic
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, copper alloy, glass Catalogued finds: AM90, AM91, AM92, AM93, AM94, AM95, AM96, AM97, AM98, AM99, AM100, AM101, AM102, AM103, AM104, AM105, AM106, AM107, AM108, AM109, AM110, AM111, AM112, AM113, AM114, C90, G48, IR371, IR415, L10, L120, PT341, PT343, PT344, PT356, PT357, PT360, PT362, PT368, PT375, PT378, PT382, PT385, PT430, PT434, PT438, PT456, Q3
----------------------
Objective 0:  AM110
Objective 1:  bone
Document b'v1ch13'
Text:  Bulk finds: Ceramic, animal bone, iron, copper alloy, glass Catalogued finds: AM90, AM91, AM92, AM93, AM94, AM95, AM96, AM97, AM98, AM99, AM100, AM101, AM102, AM103, AM104, AM105, AM106, AM107, AM108, AM109, AM110, AM111, AM112, AM113, AM114, C90, G48, IR371, IR415, L10, L120, PT341, PT343, PT344, PT356, PT357, PT360, PT362, PT368, PT375, PT378, PT382, PT385, PT430, PT434, PT438, PT456, Q3
----------------------
Objective 0:  AM110
Objective 1:  iron
Document b'v1ch1

Objective 0:  IN2
Objective 1:  stone
Document b'v1ch09'
Text:  28-9 and the middle of l. 29; letter traces on the right edge of the stone fall below IN2, l. 28.
----------------------
Objective 0:  IN1
Objective 1:  inscriptions
Document b'v1ch09'
Text:  inscriptions, with the exception of the first series texts, represented by So, Ad, cb, and IN1, and an honorific in- scription from ephesos now in the Ashmolean Museum (
----------------------
Objective 0:  IN2
Objective 1:  inscriptions
Document b'v1ch09'
Text:  The inscriptions recovered from Zeugma overlap Ancoz fragments Ana (IN2) and
----------------------
Objective 0:  IN1
Objective 1:  inscriptions
Document b'v1ch09'
Text:  The formulation of commagenian royal inscriptions is repeated for the most part without significant alteration in separate sanctuary publications, but cb and the new Ancoz fragments both offer minor variations in phrasing.34The restorations offered for IN1 below are drawn directly from So and Ad, but IN1 see

In [44]:
for i in train_cands[0:10]:
    print(i)
    char_start0 = i.get_contexts()[0].char_start
    char_end0 = i.get_contexts()[0].char_end
    char_start1 = i.get_contexts()[1].char_start
    char_end1 = i.get_contexts()[1].char_end
    print("Objective 0: ", i.get_contexts()[0].sentence.text[char_start0:char_end0+1])
    print("Objective 1: ", i.get_contexts()[0].sentence.text[char_start1:char_end1+1])
    print(i.get_contexts()[0].sentence.document)
    print("Text: ", i.get_contexts()[0].sentence.text)
#     print(i.values)
    print('----------------------')

Cata_Matl(Span("b'AM330'", sentence=993360, chars=[36,40], words=[8,8]), Span("b'Quartz'", sentence=993360, chars=[0,5], words=[0,0]))
Objective 0:  AM330
Objective 1:  Quartz
Document b'v2ch03'
Text:  Quartz range is uneven, not even as AM330.
----------------------
Cata_Matl(Span("b'G118'", sentence=954630, chars=[83,86], words=[17,17]), Span("b'faience'", sentence=954630, chars=[52,58], words=[10,10]))
Objective 0:  G118
Objective 1:  faience
Document b'v2ch08'
Text:  Even more modest was a finger ring made entirely of faience, with a conical bezel (G118).
----------------------
Cata_Matl(Span("b'G17'", sentence=948636, chars=[32,34], words=[7,7]), Span("b'glass'", sentence=948636, chars=[208,212], words=[41,41]))
Objective 0:  G17
Objective 1:  glass
Document b'v2ch08'
Text:  A fragment of a conical beaker (G17) recovered at Zeugma illustrates a decorative technique known as relief-cutting, in which the cutting wheel was used to grind down parts of the exterior surface of a blown-g

In [42]:
%%time
for i, docs in enumerate([dev_docs, test_docs]):
    candidate_extractor.apply(docs, split=i+1)
    print("Number of candidates:", session.query(Cata_Matl).filter(Cata_Matl.split == i+1).count())

[========================================] 100%
Number of candidates: 37
[========================================] 100%
Number of candidates: 0
CPU times: user 8.74 s, sys: 47.5 ms, total: 8.79 s
Wall time: 10.9 s


In [46]:
from fonduer import BatchFeatureAnnotator

featurizer = BatchFeatureAnnotator(Cata_Matl)
%time F_train = featurizer.apply(split=0, replace_key_set=True, parallelism=PARALLEL)
print(F_train.shape)
%time F_dev = featurizer.apply(split=1, replace_key_set=False, parallelism=PARALLEL)
print(F_dev.shape)
%time F_test = featurizer.apply(split=2, replace_key_set=False, parallelism=PARALLEL)
print(F_test.shape)

Copying cata_matl_feature_updates to postgres
b'COPY 3096\n'
CPU times: user 2.15 s, sys: 153 ms, total: 2.31 s
Wall time: 1min 24s
(3096, 36183)
Copying cata_matl_feature_updates to postgres
b'COPY 37\n'
CPU times: user 103 ms, sys: 103 ms, total: 206 ms
Wall time: 5.82 s
(37, 36183)


ValueError: No candidates in current split

NameError: name 'F_test' is not defined

In [ ]:
from zeugma_utils import load_hardware_labels

gold_file = 'data/zeugma_gold.csv'
load_hardware_labels(session, Cata_Labref, gold_file, ATTRIBUTE ,annotator_name='gold')